In [1]:
from google.cloud import bigquery
bq = bigquery.Client()

In [2]:
%load_ext google.cloud.bigquery

In [3]:
%%bigquery
select count(*) as num_rows 
from bigquery-public-data.geo_us_boundaries.counties

Query is running:   0%|          |

Downloading:   0%|          |

,num_rows
0,3233


In [4]:
%%bigquery
select *
from bigquery-public-data.geo_us_boundaries.counties
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,state_fips_code,county_fips_code,county_gnis_code,county_name,lsad_name,lsad_code,fips_class_code,mtfcc_feature_class_code,csa_fips_code,cbsa_fips_code,met_div_fips_code,functional_status,area_land_meters,area_water_meters,int_point_lat,int_point_lon,int_point_geom,county_geom
0,01011,01,01011,00161531,Bullock,Bullock County,06,H1,G4020,None,None,None,A,1613076054,6038093,32.101759,-85.717261,POINT(-85.7172613 32.1017589),"POLYGON((-85.926296 32.054676, -85.920111 32.0..."
1,01025,01,01025,00161538,Clarke,Clarke County,06,H1,G4020,None,None,None,A,3207494106,36657891,31.685521,-87.818624,POINT(-87.8186244 31.6855211),"POLYGON((-88.080207 31.677064, -88.080058 31.6..."
2,01027,01,01027,00161539,Clay,Clay County,06,H1,G4020,None,None,None,A,1564251834,5285207,33.270400,-85.863525,POINT(-85.8635254 33.2703999),"POLYGON((-85.978793 33.327568, -85.978797 33.3..."
3,01029,01,01029,00161540,Cleburne,Cleburne County,06,H1,G4020,None,None,None,A,1450663786,2354896,33.671964,-85.516126,POINT(-85.5161261 33.6719637),"POLYGON((-85.603465 33.788474, -85.603469 33.7..."
4,01111,01,01111,00161582,Randolph,Randolph County,06,H1,G4020,None,None,None,A,1503615937,9285420,33.296475,-85.464068,POINT(-85.4640679 33.2964753),"POLYGON((-85.648139 33.359095, -85.648165 33.3..."


In [5]:
%%bigquery water_df
select area_water_meters
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'

Query is running:   0%|          |

Downloading:   0%|          |

In [6]:
%%bigquery total_df
SELECT SUM(area_land_meters + area_water_meters) AS total_area
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'

Query is running:   0%|          |

Downloading:   0%|          |

In [7]:
#q1
water_df['area_water_meters'][0] / total_df['total_area'][0] * 100

3.314489915768649

In [8]:
%%bigquery
select *
from bigquery-public-data.geo_us_boundaries.states
limit 5

Query is running:   0%|          |

Downloading:   0%|          |

,geo_id,region_code,division_code,state_fips_code,state_gnis_code,state,state_name,lsad_code,mtfcc_feature_class_code,functional_status,area_land_meters,area_water_meters,int_point_lat,int_point_lon,int_point_geom,state_geom
0,66,9,0,66,01802705,GU,Guam,00,G4000,A,543555844,934337453,13.441745,144.771902,POINT(144.7719021 13.4417451),"POLYGON((144.563426 13.448065, 144.56355 13.44..."
1,72,9,0,72,01779808,PR,Puerto Rico,00,G4000,A,8868701898,4922576715,18.217648,-66.410799,POINT(-66.4107992 18.217648),"MULTIPOLYGON(((-67.543025 18.386123, -67.54243..."
2,69,9,0,69,01779809,MP,Commonwealth of the Northern Mariana Islands,00,G4000,A,472292529,4644252458,14.936784,145.601021,POINT(145.601021 14.9367835),"MULTIPOLYGON(((145.701166 16.724112, 145.70002..."
3,78,9,0,78,01802710,VI,United States Virgin Islands,00,G4000,A,348021896,1550236199,18.326748,-64.971251,POINT(-64.9712508 18.326748),"MULTIPOLYGON(((-64.956714 17.678977, -64.95648..."
4,60,9,0,60,01802701,AS,American Samoa,00,G4000,A,197759069,1307243751,-14.267159,-170.668267,POINT(-170.6682674 -14.267159),"MULTIPOLYGON(((-170.797156 -14.246637, -170.79..."


In [9]:
%%bigquery county_counts_df
SELECT s.state, COUNT(c.county_fips_code) AS county_count
FROM bigquery-public-data.geo_us_boundaries.counties c
JOIN bigquery-public-data.geo_us_boundaries.states s
ON c.state_fips_code = s.state_fips_code
GROUP BY s.state
ORDER BY county_count DESC
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

In [10]:
#q2
dict_list = county_counts_df.to_dict(orient='records')

county_counts_dict = {}

for entry in dict_list:
    county_counts_dict[entry['state']] = entry['county_count']

county_counts_dict

{'TX': 254, 'GA': 159, 'VA': 133, 'KY': 120, 'MO': 115}

In [11]:
no_cache = bigquery.QueryJobConfig(use_query_cache=False)

In [12]:
water = bq.query("""
select area_water_meters
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE county_name = 'Dane'
""", job_config=no_cache)

In [13]:
count = bq.query("""
SELECT s.state, COUNT(c.county_fips_code) AS county_count
FROM bigquery-public-data.geo_us_boundaries.counties c
JOIN bigquery-public-data.geo_us_boundaries.states s
ON c.state_fips_code = s.state_fips_code
GROUP BY s.state
ORDER BY county_count DESC
LIMIT 5
""", job_config=no_cache)

In [14]:
water.to_dataframe
count.to_dataframe

<bound method QueryJob.to_dataframe of QueryJob<project=cs544-412621, location=US, id=2002f5bd-902a-47c0-93b5-efe780159740>>

In [15]:
#q3
prices = {}

tb = 1024**4
price_per_tb = 6.25

prices['q1'] = water.total_bytes_billed / tb * price_per_tb
prices['q2'] = count.total_bytes_billed / tb * price_per_tb
prices

{'q1': 5.9604644775390625e-05, 'q2': 0.00011920928955078125}

In [16]:
project = "cs544-412621" 

In [17]:
ds = bigquery.Dataset(f"{project}.p8")
bq.create_dataset(ds, exists_ok=True)

Dataset(DatasetReference('cs544-412621', 'p8'))

In [18]:
config = bigquery.LoadJobConfig(source_format="PARQUET", write_disposition="WRITE_TRUNCATE")
source = "gs://s24-lukee/hdma-wi-2021.parquet"
dataset = "p8"
job = bq.load_table_from_uri(source, f"{project}.{dataset}.hdma", job_config=config)
job.result()

LoadJob<project=cs544-412621, location=US, id=bd721278-2457-4691-8102-87789bdeef71>

In [19]:
#q4
[ds.dataset_id for ds in bq.list_datasets(project)]

['lec_demo', 'p8']

In [20]:
%%bigquery
SELECT * 
FROM p8.hdma
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,state_code,county_code,census_tract,action_taken,loan_amount,interest_rate,loan_term,income,approved,dataset,index
0,WI,None,None,1,125000.0,5.95,180.0,125000.0,True,train,25517
1,WI,None,None,1,35000.0,5.95,180.0,83000.0,True,test,25575
2,WI,None,None,2,285000.0,3.79,120.0,525000.0,False,train,59936
3,WI,None,None,2,135000.0,2.79,120.0,94000.0,False,train,59982
4,WI,None,None,2,35000.0,6.29,180.0,113000.0,False,train,60035


In [21]:
%%bigquery loans_df
SELECT c.county_name, COUNT(h.action_taken) AS loan_count
FROM p8.hdma h
JOIN bigquery-public-data.geo_us_boundaries.counties c
ON h.county_code = c.county_fips_code
GROUP BY c.county_name
ORDER BY loan_count DESC
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

In [22]:
#q5
dict_list = loans_df.to_dict(orient='records')

loans_dict = {}

for entry in dict_list:
    loans_dict[entry['county_name']] = entry['loan_count']

loans_dict

{'Milwaukee': 46570,
 'Dane': 38557,
 'Waukesha': 34159,
 'Brown': 15615,
 'Racine': 13007,
 'Outagamie': 11523,
 'Kenosha': 10744,
 'Washington': 10726,
 'Rock': 9834,
 'Winnebago': 9310}

In [23]:
url = "https://docs.google.com/spreadsheets/d/1FfalqAWdzz01D1zIvBxsDWLW05-lvANWjjAj2vI4A04/"

external_config = bigquery.ExternalConfig("GOOGLE_SHEETS")
external_config.source_uris = [url]
external_config.options.skip_leading_rows = 1
external_config.autodetect = True

table = bigquery.Table(f"{project}.{dataset}.applications")
table.external_data_configuration = external_config

table = bq.create_table(table, exists_ok=True)

In [24]:
%%bigquery
SELECT *
FROM `p8.applications`
LIMIT 5

Query is running:   0%|          |

Downloading:   0%|          |

,Timestamp,loan_amount,income,latitude,longitude
0,2024-04-21 01:31:15.331000+00:00,100000,10000,43.069646,-89.753298
1,2024-04-21 01:31:36.357000+00:00,200000,20000,45.058784,-87.665404
2,2024-04-21 01:31:56.505000+00:00,200000,20000,45.000711,-87.229077
3,2024-04-21 01:31:15+00:00,150000,20000,43.006125,-88.846895
4,2024-04-21 01:31:15+00:00,210000,25000,46.815340,-90.829876


In [25]:
%%bigquery my_income_df
SELECT count(*) AS my_income_count
FROM p8.applications
WHERE income = 500000

Query is running:   0%|          |

Downloading:   0%|          |

In [26]:
#q6
my_income_df['my_income_count'][0]

9

In [27]:
%%bigquery
SELECT county_geom
FROM bigquery-public-data.geo_us_boundaries.counties
WHERE state_fips_code = '55'

Query is running:   0%|          |

Downloading:   0%|          |

,county_geom
0,"POLYGON((-90.925417 46.587301, -90.924712 46.5..."
1,"POLYGON((-87.432419 45.111772, -87.439231 45.0..."
2,"POLYGON((-91.166006 44.335102, -91.165998 44.3..."
3,"POLYGON((-90.668478 43.466273, -90.668478 43.4..."
4,"POLYGON((-92.760909 45.81664, -92.759938 45.81..."
...,...
67,"POLYGON((-89.012343 43.023489, -89.012346 43.0..."
68,"POLYGON((-90.316574 44.953278, -90.316576 44.9..."
69,"POLYGON((-90.042787 45.4676, -90.04279 45.4667..."
70,"POLYGON((-88.69812 42.842634, -88.698507 42.84..."


In [28]:
%%bigquery applications_df
SELECT c.county_name, COUNT(*) AS application_count
FROM p8.applications a
JOIN bigquery-public-data.geo_us_boundaries.counties c
ON ST_CONTAINS(c.county_geom, ST_GEOGPOINT(a.longitude, a.latitude))
WHERE c.state_fips_code = '55'
GROUP BY c.county_name

Query is running:   0%|          |

Downloading:   0%|          |

In [29]:
#q7
dict_list = applications_df.to_dict(orient='records')

applications_dict = {}

for entry in dict_list:
    applications_dict[entry['county_name']] = entry['application_count']

applications_dict

{'Douglas': 2,
 'Door': 12,
 'Dane': 57,
 'Walworth': 8,
 'Jefferson': 4,
 'Milwaukee': 8,
 'Kewaunee': 1,
 'Bayfield': 3,
 'Monroe': 2,
 'Winnebago': 3,
 'Oneida': 3,
 'Waupaca': 2,
 'Sheboygan': 3,
 'Waukesha': 1,
 'Barron': 1,
 'Fond du Lac': 1,
 'Marathon': 1,
 'Sauk': 2,
 'Manitowoc': 2,
 'Brown': 4,
 'Marinette': 3,
 'Vilas': 1,
 'Portage': 1,
 'Columbia': 2,
 'Ashland': 1,
 'Juneau': 1,
 'Green Lake': 1,
 'Outagamie': 2,
 'Florence': 1,
 'Clark': 1,
 'Iron': 1}

In [30]:
%%bigquery
CREATE OR REPLACE MODEL `p8.lr`
OPTIONS(model_type='LINEAR_REG', INPUT_LABEL_COLS=["loan_amount"]) AS
SELECT income, loan_term, loan_amount
FROM p8.hdma
WHERE dataset = 'train'

Query is running:   0%|          |

""


In [31]:
%%bigquery test
SELECT *
FROM ML.PREDICT(MODEL p8.lr, (
    select income, loan_term, loan_amount 
    from p8.hdma
    where dataset = 'test'
))

Query is running:   0%|          |

Downloading:   0%|          |

In [32]:
test

,predicted_loan_amount,income,loan_term,loan_amount
0,190869.128625,153000.0,180.0,165000.0
1,302472.158923,206000.0,360.0,275000.0
2,241796.332915,131000.0,360.0,25000.0
3,241796.332915,131000.0,360.0,225000.0
4,259594.575211,153000.0,360.0,405000.0
...,...,...,...,...
69950,191637.650082,69000.0,360.0,235000.0
69951,191637.650082,69000.0,360.0,245000.0
69952,191637.650082,69000.0,360.0,215000.0
69953,191637.650082,69000.0,360.0,145000.0


In [33]:
#q8
mae = 0
for index, row in test.iterrows():
    mae += abs(row['predicted_loan_amount'] - row['loan_amount'])
mae /= len(test)
mae

77294.6840846409

In [34]:
%%bigquery weights
SELECT *
FROM ML.WEIGHTS(MODEL p8.lr)

Query is running:   0%|          |

Downloading:   0%|          |

In [35]:
weights

,processed_input,weight,category_weights
0,income,0.809011,[]
1,loan_term,381.808037,[]
2,__INTERCEPT__,-1635.003017,[]


In [36]:
#q9
weights['weight'][1]

381.80803658769094

In [37]:
%%bigquery form
SELECT loan_amount, income
FROM p8.applications

Query is running:   0%|          |

Downloading:   0%|          |

In [38]:
form

,loan_amount,income
0,100000,10000
1,200000,20000
2,200000,20000
3,150000,20000
4,210000,25000
...,...,...
209,400000,200000
210,3000000,100000
211,200000,120000
212,320000,150000


In [39]:
#q10
count = 0
for index, row in form.iterrows():
    predicted = weights['weight'][0] * row['income'] + weights['weight'][1] * 30 + weights['weight'][2]
    actual = row['loan_amount']
    if actual > predicted:
        count += 1

ratio = count / len(form)
ratio

0.7757009345794392